In [1]:
import sys
import os
TOP = "C:/Users/vishk/Desktop/WIP/2020/2020 Q1/07 - Self Learning"
sys.path.append(f"{TOP}/lib")
os.environ["TOP"] = TOP
from bootstrap import *

Using TensorFlow backend.


In [2]:
pd.plotting.register_matplotlib_converters()
mysql_con, sql_svr_con = get_con(cfg['mysql'], cfg['sql_svr'])
con = sql_svr_con

In [3]:
def cr_us_mds(us_tbl):
    id_vars = [
        'UID',
        'FIPS',
        'Province_State',
        'Country_Region',
        'Lat',
        'Long_',
        'Combined_Key'
    ]

    death_id_vars = id_vars + [ 'Population' ]

    drop_cols = [
        'iso2',
        'iso3',
        'code3',
        'Admin2'
    ]

    rename_cols = {
        'UID':            'uid',
        'FIPS':           'fips',
        'Province_State': 'state',
        'Country_Region': 'country',
        'Lat':            'lat',
        'Long_':          'long',
        'Combined_Key':   'town',
        'Population':     'population'
    }

    if 'death' in us_tbl:
        id_vars_to_use = death_id_vars
        value_name = 'deaths'
    else:
        id_vars_to_use = id_vars
        value_name = 'incidence'

    sql = f"select * from {us_tbl}"
    df = pd.read_sql(sql, con)

    df.drop(columns=drop_cols, axis=1, inplace=True)
    df_unpivoted = df.melt(id_vars=id_vars_to_use, var_name ='date', value_name = value_name)
    df_unpivoted['date'] = df_unpivoted['date'].astype('datetime64[ns]')
    df_unpivoted.sort_values(by=['Combined_Key', 'date'], inplace=True)
    df_unpivoted.rename(columns=rename_cols, inplace=True)
    df_to_sql(df_unpivoted, f"{us_tbl}_mds", con)
    
    sql = f"select * from {us_tbl}_mds"
    df = pd.read_sql(sql, con)
    display(df)
    
    return df


In [4]:
def cr_raw_mds():
    us_tbls = [ 
        'covid19_us_cases',
        'covid19_us_deaths',
    ]

    for us_tbl in us_tbls:
        cr_us_mds(us_tbl)
        
    return None

In [5]:
def cr_us_mds_fin():
    us_tbl_mds_raw = 'covid19_us_mds_raw'
    us_tbl_mds     = 'covid19_us_mds'
    
    sql = f"""
        drop table if exists {us_tbl_mds_raw};
        
        select
            a.uid,
            a.town,
            a.[date],
            a.state,
            a.country,
            a.fips,
            a.lat,
            a.long,
            b.population,
            a.incidence,
            b.deaths,
            case 
                when b.population > 0 then round((a.incidence+0.0)*100/b.population,2) 
                else 0
            end as incidence_rate_pct,
            case 
                when a.incidence > 0 then round((b.deaths+0.0)*100/a.incidence,2)
                else 0
            end as death_rate_pct
        into
            {us_tbl_mds_raw}
        from
            covid19_us_cases_mds a
            left join
            covid19_us_deaths_mds b
            on
            a.uid = b.uid and a.[date] = b.[date]
        ;
        
        drop table if exists {us_tbl_mds};
        select
            uid,
            town,
            [date],
            state,
            country,
            fips,
            lat,
            long,
            population,
            incidence_rate_pct,
            incidence,
            incidence_lg1,
            incidence - incidence_lg1 as incidence_inc,
            case
                when incidence_lg1 > 0 then round(((incidence - incidence_lg1)+0.0)*100/incidence_lg1, 2)
                when incidence > 0 and incidence_lg1 = 0 then 100
                else 0
            end as incidence_inc_pct,
            
            death_rate_pct,
            deaths,
            deaths_lg1,
            deaths    - deaths_lg1    as deaths_inc,
            case
                when deaths_lg1 > 0 then round(((deaths - deaths_lg1)+0.0)*100/deaths_lg1, 2)
                when deaths > 0 and deaths_lg1 = 0 then 100
                else 0
            end as death_inc_pct
        into
            {us_tbl_mds}
        from
        (
            select
                a.*,

                lag(incidence,1) over (
                    partition by
                        uid

                    order by
                        [date]
                ) as incidence_lg1,
                
                lag(deaths,1) over (
                    partition by
                        uid

                    order by
                        [date]
                ) as deaths_lg1
            from
                {us_tbl_mds_raw} a
        )b;
    """
    con.execute(sql)
    return None


In [6]:
cr_raw_mds()
cr_us_mds_fin()

uid     fips           state country        lat        long  \
0       84045001  45001.0  South Carolina      US  34.223334  -82.461707   
1       84045001  45001.0  South Carolina      US  34.223334  -82.461707   
2       84045001  45001.0  South Carolina      US  34.223334  -82.461707   
3       84045001  45001.0  South Carolina      US  34.223334  -82.461707   
4       84045001  45001.0  South Carolina      US  34.223334  -82.461707   
...          ...      ...             ...     ...        ...         ...   
273247  84046137  46137.0    South Dakota      US  44.978819 -101.665462   
273248  84046137  46137.0    South Dakota      US  44.978819 -101.665462   
273249  84046137  46137.0    South Dakota      US  44.978819 -101.665462   
273250  84046137  46137.0    South Dakota      US  44.978819 -101.665462   
273251  84046137  46137.0    South Dakota      US  44.978819 -101.665462   

                                 town       date  incidence  
0       Abbeville, South Carolina, US 2020-01-22          0  
1       Abbeville, South Carolina, US 2020-01-23          0  
2       Abbeville, South Carolina, US 2020-01-24          0  
3       Abbeville, South Carolina, US 2020-01-25          0  
4       Abbeville, South Carolina, US 2020-01-26          0  
...                               ...        ...        ...  
273247      Ziebach, South Dakota, US 2020-04-10          0  
273248      Ziebach, South Dakota, US 2020-04-11          0  
273249      Ziebach, South Dakota, US 2020-04-12          0  
273250      Ziebach, South Dakota, US 2020-04-13          0  
273251      Ziebach, South Dakota, US 2020-04-14          0  

[273252 rows x 9 columns]

uid     fips           state country        lat        long  \
0       84045001  45001.0  South Carolina      US  34.223334  -82.461707   
1       84045001  45001.0  South Carolina      US  34.223334  -82.461707   
2       84045001  45001.0  South Carolina      US  34.223334  -82.461707   
3       84045001  45001.0  South Carolina      US  34.223334  -82.461707   
4       84045001  45001.0  South Carolina      US  34.223334  -82.461707   
...          ...      ...             ...     ...        ...         ...   
273247  84046137  46137.0    South Dakota      US  44.978819 -101.665462   
273248  84046137  46137.0    South Dakota      US  44.978819 -101.665462   
273249  84046137  46137.0    South Dakota      US  44.978819 -101.665462   
273250  84046137  46137.0    South Dakota      US  44.978819 -101.665462   
273251  84046137  46137.0    South Dakota      US  44.978819 -101.665462   

                                 town  population       date  deaths  
0       Abbeville, South Carolina, US       24527 2020-01-22       0  
1       Abbeville, South Carolina, US       24527 2020-01-23       0  
2       Abbeville, South Carolina, US       24527 2020-01-24       0  
3       Abbeville, South Carolina, US       24527 2020-01-25       0  
4       Abbeville, South Carolina, US       24527 2020-01-26       0  
...                               ...         ...        ...     ...  
273247      Ziebach, South Dakota, US        2756 2020-04-10       0  
273248      Ziebach, South Dakota, US        2756 2020-04-11       0  
273249      Ziebach, South Dakota, US        2756 2020-04-12       0  
273250      Ziebach, South Dakota, US        2756 2020-04-13       0  
273251      Ziebach, South Dakota, US        2756 2020-04-14       0  

[273252 rows x 10 columns]

In [7]:
sql = f"select * from covid19_us_mds"
df = pd.read_sql(sql, con)
curr_date = datetime.now().date()
#df_to_excel(df,"covid19_2020_04_05.xlsx")
df.to_csv(f"covid19_us_{curr_date}.csv")

In [8]:

#print(curr_date)